In [ ]:
"Predicting Natural Gas Prices Using Machine Learning:"

In [ ]:
"Alyssa Robinson - ITCS-5156-051"

In [2]:
# Install and Import required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# This dataset is from the Kaggle Repository and can be downloaded at: "https://www.kaggle.com/datasets/tunguz/natural-gas-prices"

In [18]:
# Load the daily gas price dataset 
df_daily = pd.read_csv('daily_csv.csv')
display(df_daily)

In [ ]:
# Get a summary of the data
df_daily_describe = df_daily.describe()
df_daily

In [ ]:

# Check for null values
df_daily.isnull().any()

df_daily['Price'].fillna(df_daily['Price'], inplace=True)

In [ ]:
from sklearn.impute import SimpleImputer
# Data Cleaning
df_daily_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
df_daily_mean.fit(df_daily)
df_daily_mean.transform(df_daily)

display(df_daily_mean)
np.any(df_daily_mean.isna())

In [ ]:
# Visualize the Dataset:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12,12))
plt.scatter(df_daily['day'], df_daily['Price'], color='green')
plt.xlabel('Day')
plt.ylabel('Price')
plt.legend()
plt.show()


In [ ]:
import seaborn as sns
sns.lineplot(x='day', y='Price', data =df_daily, color='blue')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Train_test_split is used to split the dataset into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = DecisionTreeClassifier()

model.fit(X_train, y_train)

train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)
print("Train Accuracy: {}, Test Accuracy: {}".format(train_score, test_score))

In [ ]:
daily_predict = model.predict(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=6)  # 6 trees as the figure

rf.fit(X_train, y_train)

train_score = rf.score(X_train, y_train)
test_score = rf.score(X_test, y_test)
print("Train Accuracy: {}, Test Accuracy: {}".format(train_score, test_score))

In [ ]:
rf.estimators_